In [15]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import itertools

########################


# Data

def load_csv(file_path):
    # CSV 파일 로드 (첫 번째 행을 건너뜀, 앞에서부터 8개의 열만 불러옴)
    data = np.genfromtxt(file_path, delimiter=',', skip_header=1, usecols=range(8))

    # 'HAEMOGLOBINS'를 목표 변수로 설정 (두 번째 열)
    x_data = np.delete(data, 1, axis=1)  # 두 번째 열 제거 (입력 변수)
    y_data = data[:, 1].reshape(-1, 1)  # 두 번째 열이 목표 변수
    
    return x_data, y_data

def split_data(x_data, y_data):
    # 데이터셋을 학습, 검증, 테스트로 나누기
    # random_state는 Seed값으로 동일한 분할 결과를 얻기 위해 사용
    # 전체 데이터의 20%를 테스트 데이터로 사용
    x_temp, x_test, y_temp, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    # 테스트 데이터를 제외한 나머지 데이터의 12.5%를 검증 데이터로 사용, 
    # 나머지를 학습 데이터로 사용
    x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.125, random_state=42) # stratify=y_data가 없음

    # 표준화 (스케일링)
    sc = StandardScaler()
    # 학습 데이터에 fit_transform을 사용하여 평균과 표준편차를 계산하고 데이터를 표준화
    x_train = sc.fit_transform(x_train)
    # 검증 및 테스트 데이터에는 학습 데이터의 평균과 표준편차를 사용하여 동일하게 표준화
    x_val = sc.transform(x_val)
    x_test = sc.transform(x_test)
    
    return x_train, x_val, x_test, y_train, y_val, y_test

########################
# Learning

def create_graph(optimizer, input_num, hidden_sizes):
    tf.reset_default_graph()  # 그래프 초기화
    
    x = tf.placeholder(tf.float32, [None, input_num], name='x')  # 입력 데이터 플레이스홀더
    y = tf.placeholder(tf.float32, [None, 1], name='y')  # 레이블 데이터 플레이스홀더 (회귀 문제)

    # 히든 레이어 생성
    layer_input = x
    layer_input_size = input_num
    for idx, hidden_size in enumerate(hidden_sizes):
        # 가중치 및 바이어스 초기화
        # tf.random.normal - 정규 분포를 따르는 랜덤 값 생성, 가중치 초기화용
        W_hidden = tf.Variable(tf.random.normal([layer_input_size, hidden_size], stddev=0.1), name=f'W_hidden_{idx+1}')
        b_hidden = tf.Variable(tf.zeros([hidden_size]), name=f'b_hidden_{idx+1}')
        # 레이어 출력 계산
        layer_output = tf.nn.relu(tf.matmul(layer_input, W_hidden) + b_hidden, name=f'relu_{idx+1}')

       
        # 다음 레이어를 위해 업데이트
        layer_input = layer_output
        layer_input_size = hidden_size

    # 출력 레이어 (1개의 출력값)
    W_output = tf.Variable(tf.random.normal([layer_input_size, 1], stddev=0.1), name='W_output')
    b_output = tf.Variable(tf.zeros([1]), name='b_output')

    prediction = tf.matmul(layer_input, W_output) + b_output  # 예측값 계산
    loss = tf.reduce_mean(tf.square(y - prediction))  # MSE 손실 계산
    train_step = optimizer.minimize(loss)  # 최적화 수행

    return x, y, prediction, loss, train_step

def execute_graph(x, y, prediction, loss, train_step, data, epoch, batch_size):
    train_losses = []
    val_losses = []
    test_losses = []

    x_train, x_val, x_test, y_train, y_val, y_test = data
   
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())  # 변수 초기화
        for e in range(epoch):
            # 학습 데이터 셔플
            permutation = np.random.permutation(len(x_train))
            x_train_shuffled = x_train[permutation]
            y_train_shuffled = y_train[permutation]

            num_batches = (len(x_train) + batch_size - 1) // batch_size
            
            for i in range(num_batches):
                start_idx = i * batch_size
                end_idx = min((i+1) * batch_size, len(x_train))
                batch_x = x_train_shuffled[start_idx:end_idx]
                batch_y = y_train_shuffled[start_idx:end_idx]
                sess.run(train_step, feed_dict={x: batch_x, y: batch_y})

            if e % 10 == 1:
                # 손실 계산
                train_loss = sess.run(loss, feed_dict={x: x_train, y: y_train})
                val_loss = sess.run(loss, feed_dict={x: x_val, y: y_val})
                test_loss = sess.run(loss, feed_dict={x: x_test, y: y_test})

                train_losses.append(train_loss)
                val_losses.append(val_loss)
                test_losses.append(test_loss)
                print(f'Epoch {e}, Training Loss: {train_loss}, Validation Loss: {val_loss}, Test Loss: {test_loss}')
       
        # 최종 예측 수행
        predictions = sess.run(prediction, feed_dict={x: x_test})

    return train_losses, val_losses, test_losses, predictions

def learn(data, param_grid):

    x_train = data[0]
    best_report = None

    # 하이퍼파라미터 조합 생성
    keys = param_grid.keys()
    combinations = list(itertools.product(*(param_grid[key] for key in keys)))

    print('1. Training Model')
    print(f"Total parameter combinations: {len(combinations)}")

    for idx, combo in enumerate(combinations, 1):
        params = dict(zip(keys, combo))
        print(f'Training model {idx}/{len(combinations)} with parameters:')
        print(f"Hidden sizes: {params['hidden_sizes']}, Optimizer: 	{params['optimizer'].__name__}, Learning rate: 	{params['learning_rate']}, Batch size: {params['batch_size']}, 	Epochs: {params['epoch']}")

        optimizer_class = params['optimizer']
        learning_rate = params['learning_rate']
        hidden_sizes = params['hidden_sizes']
        epoch = params['epoch']
        batch_size = params['batch_size']

                            
        optimizer = optimizer_class(learning_rate=learning_rate)

        # 그래프 생성
        x, y, prediction, loss, train_step = create_graph(
            optimizer, x_train.shape[1], hidden_sizes)
                
        # 그래프 실행
        train_losses, val_losses, test_losses, predictions = execute_graph(
            x, y, prediction, loss, train_step, data, epoch, batch_size)

        # 결과 출력
        print(f"Optimizer: {optimizer_class.__name__}, Learning rate: {learning_rate}")
        print(f"Hidden sizes: {hidden_sizes}, Batch size: {batch_size}, Epochs: {params['epoch']}")
        print(f"Training loss: {train_losses[-1]:.4f}")
        print(f"Validation loss: {val_losses[-1]:.4f}")
        print(f"Test loss: {test_losses[-1]:.4f}\n")

        # 최적의 모델 업데이트 (여기서는 테스트 손실이 가장 작은 모델 선택)
        if best_report is None or test_losses[-1] < best_report['test_losses'][-1]:
            best_report = {
                "params": params,
                "train_losses": train_losses,
                "val_losses": val_losses,
                "test_losses": test_losses,
                "predictions": predictions,
                "y_test": data[-1]
            }

    return best_report

########################
# Report

def generate_report(report):
    if report is None:
        print("No report to generate.")
        return

    params = report['params']
    print(f'2. Conclusion')  # 대제목 출력
    print(f"Best Parameters:")
    print(f"Optimizer: {params['optimizer'].__name__}")
    print(f"Learning rate: {params['learning_rate']}")
    print(f"Hidden sizes: {params['hidden_sizes']}")
    print(f"Batch size: {params['batch_size']}")
    print(f"Epochs: {params['epoch']}")
    print(f"Training loss: {report['train_losses'][-1]:.4f}")
    print(f"Validation loss: {report['val_losses'][-1]:.4f}")
    print(f"Test loss: {report['test_losses'][-1]:.4f}")

    # 실제 값과 예측 값 출력
    y_test = report['y_test']
    predictions = report['predictions']
    print("\nActual Values vs Predicted Values:")
    for actual, predicted in zip(y_test.flatten(), predictions.flatten()):
        print(f"Actual: {actual:.4f}, Predicted: {predicted:.4f}")

########################
# 파라미터 그리드 설정
                            
# 3(hidden_sizes) * 3(optimizer) * 3(learning_rate) * 3(batch_size) * 3(epochs) = 243 개의 모델을 생성
# 뉴런 수 (Ex. [32, 16, 8]로 설정하면 3개의 히든 레이어가 생성 되고 뉴런 수는 각각 32, 16, 8로 할당)
param_grid = {
    'hidden_sizes': [[32, 16, 8], [16, 16, 16], [8, 16, 32]],
    'optimizer': [tf.train.GradientDescentOptimizer, tf.train.AdamOptimizer, tf.train.AdagradOptimizer],
    'learning_rate': [0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64],
    'epoch': [100, 200, 300]
}

########################
# 메인 실행 흐름

def main():
    # Data
    x_data, y_data = load_csv('data-ori.csv')
    x_train, x_val, x_test, y_train, y_val, y_test = split_data(x_data, y_data)
    data = (x_train, x_val, x_test, y_train, y_val, y_test)

    # Learning
    best_report = learn(data, param_grid)

    # Report
    generate_report(best_report)
    
if __name__ == "__main__":
    main()


1. Training Model
Total parameter combinations: 243
Training model 1/243 with parameters:
Hidden sizes: [32, 16, 8], Optimizer: 	GradientDescentOptimizer, Learning rate: 	0.01, Batch size: 16, 	Epochs: 100
Epoch 1, Training Loss: 0.03020770661532879, Validation Loss: 0.02284235693514347, Test Loss: 0.02800462394952774
Epoch 11, Training Loss: 0.007498438935726881, Validation Loss: 0.007506952155381441, Test Loss: 0.011081351898610592
Epoch 21, Training Loss: 0.0033962393645197153, Validation Loss: 0.0036478086840361357, Test Loss: 0.004600758198648691
Epoch 31, Training Loss: 0.005837236996740103, Validation Loss: 0.005494697950780392, Test Loss: 0.006373272277414799


KeyboardInterrupt: 